In [1]:
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from datetime import date,datetime,timedelta
import time
import sys
import os
import pandas as pd

In [2]:
# KOBIS (일별 박스오피스) 페이지 접속하기
path = "C:\\web_driver\\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
time.sleep(3)

C:\Users\user\AppData\Local\Temp\ipykernel_23496\3214305788.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [3]:
# 날짜 입력 후 조회하기
start_date = date(2020, 10, 20)
end_date = date(2022, 10, 20)

while start_date <= end_date :
    try :
        time.sleep(2)
        # 시작날짜 입력
        date_from=driver.find_element(By.XPATH,'//*[@id="sSearchFrom"]')

        for i in range(10):
            date_from.send_keys(Keys.BACKSPACE) 

        date_from.send_keys(start_date.strftime("%Y-%m-%d")) 

        # 종료날짜 입력
        date_to=driver.find_element(By.XPATH,'//*[@id="sSearchTo"]')

        for i in range(10):
            date_to.send_keys(Keys.BACKSPACE)

        date_to.send_keys(start_date.strftime("%Y-%m-%d"))
        
        # 조회 버튼 클릭
        search_btn=driver.find_element(By.XPATH,'//*[@id="searchForm"]/div/div[5]/button')
        search_btn.click()
        time.sleep(1)
        
        start_date+=timedelta(days=1)
        
        #더보기 클릭하여 전체 자료보기
        while True :
            try :
                driver.find_element(By.LINK_TEXT,"더보기").click()
            except :
                break
                       
        #전체 차트 수집하기         
        # 페이지 읽어오기       
        full_html=driver.page_source
        soup=BeautifulSoup(full_html, 'html.parser')

        # 검색날짜 
        search_date=soup.find('div','board_tit').get_text().replace("\n","")

        # 검색건수
        search_total=soup.find('em','fwb').get_text()
        print(search_total)
        
        # 컬럼 리스트 생성
        content_column=soup.find('thead').find_all('th',{'scope':'col'})

        column_list=[]
        column_list.append('일자')

        for i in content_column :
            column_list.append(i.text.replace("\t","").replace("\n","").replace("오름차순내림차순",""))
        print(column_list)
        
        # 항목 리스트 생성
        rank_table=soup.find('tbody').find_all('tr')

        rank_list=[]
        
        #리스트에 항목 담기
        for movie in rank_table :
            chart=[]
            chart.append(search_date)

            #영화 순위
            movie_rank=movie.find('td').text.replace("\t","").replace("\n","")
            chart.append(movie_rank)

            #영화제목
            movie_title=movie.find('span','ellip per90').get_text()
            chart.append(movie_title)
            
            #개봉일
            movie_chart=movie.select('td:nth-child(3)')
            for i in movie_chart :
                chart.append(i.text.replace("\n","").replace("\t","").replace(" ",""))

            #'매출액', '매출액점유율', '매출액증감(전일대비)', '누적매출액', '관객수' 등
            movie_chart=movie.find_all('td','tar')

            for i in movie_chart :
                chart.append(i.text.replace("\t","").replace("\n",""))

            rank_list.append(chart)
            
        print(rank_list)
        
         # 영화별 세부정보 수집하기
        movie_name_list=soup.find_all('span','ellip per90')

        movie_info_cont=[]
        movie_staff=[]
        for i in movie_name_list :
            try :
                name=i.get_text()
                time.sleep(1)

                # 영화 정보 팝업 띄우기
                movie_btn=driver.find_element(By.LINK_TEXT,str(name))
                movie_btn.click()
                time.sleep(1)

                #영화 정보 가져오기
                movie_html=driver.page_source
                soup=BeautifulSoup(movie_html, 'html.parser')

                #영화제목
                movie_name=soup.find('div','hd_layer').find('strong').get_text()

                #영화세부정보contents
                movie_contents=soup.find('dl',class_='ovf cont').find_all('dd')
                
                movie_info=[]
                movie_info.append(movie_name)
            
                for cont in movie_contents :
                    movie_info.append(cont.text.replace("\n","").replace("\t","").replace(" ","").strip())
                
                movie_info_cont.append(movie_info)
                
                #감독, 배우 정보 수집
                try : 
                    movie_staff_dict={'감독' : [],'배우' : []}

                    director=soup.find('div','staffMore').find('dd').text.strip()
                    movie_staff_dict['감독'].append(director)

                    actor_list=soup.find('div','staffMore').find('td').find_all('a')

                    for i in actor_list :
                        movie_staff_dict['배우'].append(i.text.strip())
                        
                except : 
                    movie_staff_dict={'감독' : [],'배우' : []}
                    
                movie_staff.append(movie_staff_dict)

                # 팝업 창 닫기
                time.sleep(1)
                driver.find_element(By.LINK_TEXT,"뒤로").click()
                
            except : 
                print("오류 항목을 제외합니다.")
                continue
                
        print(movie_info_cont)
        print(movie_staff)
            
    except :
        print("데이터를 수집하지 못하였습니다.")
        break

108
['일자', '순위', '영화명', '개봉일', '매출액', '매출액점유율', '매출액증감(전일대비)', '누적매출액', '관객수', '관객수증감(전일대비)', '누적관객수', '스크린수', '상영횟수']
[['2020년 10월 20일(화)', '1', '소리도 없이', '2020-10-15', '171,664,030', '35.9%', '-12,431,810(-6.8%)', '2,331,008,630', '20,730', '-1,275(-5.8%)', '263,168', '1,000', '3,378'], ['2020년 10월 20일(화)', '2', '담보', '2020-09-29', '119,240,190', '24.9%', '-7,156,030(-5.7%)', '12,779,963,140', '14,600', '-494(-3.3%)', '1,478,480', '935', '2,694'], ['2020년 10월 20일(화)', '3', '테넷', '2020-08-26', '27,094,680', '5.7%', '1,498,310(5.9%)', '17,684,765,980', '3,021', '217(7.7%)', '1,917,434', '297', '471'], ['2020년 10월 20일(화)', '4', '삼진그룹 영어토익반', '2020-10-21', '12,858,000', '2.7%', '7,477,000(139.0%)', '56,286,000', '2,587', '1,253(93.9%)', '8,787', '22', '22'], ['2020년 10월 20일(화)', '5', '그대, 고맙소 : 김호중 생애 첫 팬미팅 무비', '2020-09-29', '59,025,500', '12.3%', '13,318,000(29.1%)', '2,800,026,500', '1,993', '428(27.3%)', '90,821', '93', '157'], ['2020년 10월 20일(화)', '6', '돌멩이', '2020-10-15', '9,490,01

오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.
오류 항목을 제외합니다.데이터를 수집하지 못하였습니다.


In [4]:
#naver 영화리뷰 페이지 연결
path = "C:\\web_driver\\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://movie.naver.com/movie/point/af/list.naver")
time.sleep(3)

C:\Users\user\AppData\Local\Temp\ipykernel_23496\3091693911.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [5]:
print("----------------------------------------------------------------")

print("< NAVER 영화리뷰 크롤링을 시작합니다.>")
print()
full_html=driver.page_source    
soup=BeautifulSoup(full_html,'html.parser')

##게시물 수
content_num=soup.find_all('div','h5_right_txt')

for i in content_num:
    txt2=i.get_text().strip()
print(txt2)


#추출데이터 누적할 변수
movie_num_txt=[]
movie_name_txt=[]
review_txt=[]
list_netizen_score_txt=[]
whole_score_txt=[]
date_txt=[]


num=0

#데이터 추출
for page in range(1,2000):
    
    try :     
        full_html=driver.page_source    
        soup=BeautifulSoup(full_html,'html.parser')

        ##리뷰 번호
        movie_num=soup.find_all('td','ac num')

        for i in movie_num:
            txt=i.get_text().strip()
            num+=1
            movie_num_txt.append(txt)

        ##영화 이름
        movie_name=soup.find_all('a','movie color_b')

        for i in movie_name:
            txt=i.get_text().strip()
            num+=1
            movie_name_txt.append(txt)


        ##리뷰
        reviews = soup.find_all("td",{"class":"title"})
        for i in reviews:
            sentence = i.find("a",{"class":"report"}).get("onclick").split("', '")[2]
            #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
            # if sentence != "":
            review_txt.append([sentence])
        

        ##별점
        list_netizen_score=soup.find_all('div','list_netizen_score')

        for i in list_netizen_score:
            txt=i.get_text().strip()
            new_txt = txt.replace('별점 - 총 10점 중', '')
            num+=1
            list_netizen_score_txt.append(new_txt)

        ##총 별점
        whole_score='10'
        for i in list_netizen_score:
            num+=1
            whole_score_txt.append(whole_score)
        

        ##날짜
        date=soup.select("td:nth-child(3)")
        for i in date:
            txt=i.get_text().strip()[-8:]

            num+=1
            date_txt.append(txt)


        ##페이지 넘기기
        page+=1    
        driver.find_element(By.LINK_TEXT, str(page)).send_keys(Keys.ENTER)        

        time.sleep(3)
        
    except :
        print("페이지에 오류가 발생하였습니다.")
        continue
    
print(movie_num_txt)
print(movie_name_txt)
print(review_txt)
print(list_netizen_score_txt)
print(whole_score_txt)
print(date_txt)

#총 리뷰 개수만 추출
txt3=txt2.replace('총', '').replace('개의 평점이 있습니다.', '')
print("<총",txt3,"건 수집완료 되었습니다.>")
print()
print("<NAVER 리뷰 크롤링이 완료되었습니다.>")
print()
print("----------------------------------------------------------------")

----------------------------------------------------------------
< NAVER 영화리뷰 크롤링을 시작합니다.>

총 14,353,939개의 평점이 있습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페

페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발

페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발

페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발생하였습니다.
페이지에 오류가 발

In [6]:
import requests
from bs4 import BeautifulSoup
import sqlite3 as sql
import pymysql
import re

In [7]:
#네이버 리뷰 리스트-----------------------------------------------------------------------------
gid_n=[]
gdate_n=[]
gpoint=[]
gsumpoint=[]

for i in range(len(movie_num_txt)):  #번호
    gid_n.append(i+1)

for i in date_txt:  #날짜
    gdate_n.append('20'+i[-8:].replace('.','-'))

for i in list_netizen_score_txt:  #별점
    gpoint.append(int(i))

for i in whole_score_txt:  #총점
    gsumpoint.append(int(i))

gmovie_n=movie_name_txt  #영화제목
greview=review_txt  #리뷰
gnum=movie_num_txt  #리뷰번호(식별코드)


#코비스 리스트---------------------------------------------------------------------------------------------
gid_k=[]
gmovie_k=[]
gdate_k=[]

gaudience=[]
ggenre=[]
gopendate=[]
gdirec=[]
gactor=[]
gmovie_id=[]

gdate1=[]
gdate2=[]
ggenre1=[]

for i in range(len(rank_list)):
    gid_k.append(i+1)  #번호
    gmovie_k.append(rank_list[i][2])  #영화제목
    gaudience.append(rank_list[i][8])  #관객수
    gopendate.append(rank_list[i][3])  #개봉일
    gdate1.append(rank_list[i][0])
    gdate2=re.findall('\d+',gdate1[i])
    gdate_k.append('-'.join(gdate2))  #날짜

for i in range(len(movie_info_cont)):
    gmovie_id.append(movie_info_cont[i][1])  #코드(식별코드)
    ggenre1.append(movie_info_cont[i][4].split('|'))
    ggenre.append(ggenre1[i][2])  #장르

for i in movie_staff:
    gdirec.append(str(list(i.values())[0])[1:-1].replace("'",""))  #감독
    gactor.append(str(list(i.values())[1])[1:-1].replace("'",""))  #주연배우


#db생성------------------------------------------------------------------------------------------------------
conn = pymysql.connect(host='localhost',user='root',password='1234',charset='utf8')
curs=conn.cursor(pymysql.cursors.DictCursor)
curs.execute('create database film_innerside')
curs.execute('use film_innerside')

#네이버리뷰 가더 테이블 생성--------------------------------------------------------------------------------------
curs.execute('''create table `naver_review_gather`(
    gid			INT 		AUTO_INCREMENT,
    gmovie		VARCHAR(40)	Not Null,
    gdate		DATE,
    greview		VARCHAR(1100),
    gpoint		INT(2)	CHECK(gpoint >= 0 AND gpoint <= 10),
    gsumpoint		FLOAT(5),
    gnum		VARCHAR(10)	Not Null	UNIQUE,
    primary key(gid)
)''')

#코비스 가더 테이블 생성-----------------------------------------------------------------------------------------------------
curs.execute('''create table `kobis_movie_gather`(
    gid			INT 	        AUTO_INCREMENT,
    gmovie		VARCHAR(40)	Not Null,
    gdate		DATE		Not Null,
    gaudience	        INT(10)		Not Null,
    ggenre		VARCHAR(30),
    gopendate	        DATE,
    gdirec		VARCHAR(30),
    gactor		VARCHAR(600),
    gmovie_id	        VARCHAR(110)	Not Null	UNIQUE,
    primary key(gid)
)''')

#네이버리뷰 스크럽 테이블 생성-------------------------------------------------------------------------------------------------
curs.execute('''CREATE TABLE `naver_review_scrub`(
    sid		INT		auto_increment,
    smovie	VARCHAR(40),		
    sdate	DATE,				
    sreview	VARCHAR(1100),			
    spoint	INT(2)	CHECK(spoint >= 0 AND spoint <= 10),
    ssumpoint	FLOAT(5),
    snum	VARCHAR(10)	not null	UNIQUE,
    label	INT(1),
    positive	FLOAT(20),
    primary key(sid)
)''')

#코비스 스크럽 테이블 생성-----------------------------------------------------------------------------------------------------
curs.execute('''CREATE TABLE `kobis_movie_scrub`(
    sid		INT		auto_increment,
    smovie	VARCHAR(40)	not null,
    sdate	DATE		not null,
    saudience	INT(10)		not null,
    sgenre	VARCHAR(30),		
    sopendate	DATE,			
    sdirec	VARCHAR(30),		
    sactor	VARCHAR(600),		
    smovie_id	VARCHAR(110)	not null	UNIQUE,	
    primary key(sid)
)''')

#네이버리뷰 테이블에 넣기
for a,b,c,d,e,f,g in zip(gid_n,gmovie_n,gdate_n,greview,gpoint,gsumpoint,gnum):
    sql_n='INSERT INTO naver_review_gather VALUES(%s,%s,%s,%s,%s,%s,%s)'
    val_n=(a,b,c,d,e,f,g)
    curs.execute(sql_n,val_n)

#코비스 테이블에 넣기
for a,b,c,d,e,f,g,h,i in zip(gid_k,gmovie_k,gdate_k,gaudience,ggenre,gopendate,gdirec,gactor,gmovie_id):
    sql_k='INSERT INTO kobis_movie_gather VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    val_k=(a,b,c,d,e,f,g,h,i)
    curs.execute(sql_k,val_k)

conn.commit()
conn.close()

DataError: (1406, "Data too long for column 'greview' at row 1")